# **RI - Recuperação de Informação - Projeto 1**

# grupo:


*   André Luis - alps2
*   João Plácido - jpsn
*   Pedro Milet - pmm2

# Domínio/Tópico: Jogos de midia fisica








# **Crawler**





# **Instalando dependencias**

In [1]:
!pip install beautifulsoup4
!pip install validators

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **iniciar variaveis e imports**

In [2]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import urllib.robotparser as urobot
import concurrent.futures
import urllib.error
import validators
import socket
import time
import re
#lista de dominios a serem coletados
urls = [
            "https://www.bigboygames.com.br/",
            "https://www.mercadolivre.com.br/",
            "https://www.shopb.com.br/",
            "https://www.shockgames.com.br/",
            "https://www.vnsgames.com.br/",
            "https://www.ibyte.com.br/"
    ]

delay = 1 # tempo em segundos parar fazer uma requisição

qt_paginas = 10 #quantidades de paginas a serem coletadas por dominio 

# **Fronteira**

In [3]:
#fronteira busca em largura
class Frontier:
    def __init__(self, root):
        self.list = [root]
        self.looked = set()

    def get_next_url(self):
        nxt = self.list.pop(0)
        self.looked.add(nxt)
        return nxt

    def add_urls(self, urls):
      for url in urls:
        if url not in self.looked:
          self.list.append(url)


#fronteira com euristica Inlink
class FrontierInlink:

    def __init__(self, root):
        self.list = dict()
        self.list[root] = 0
        self.looked = set()

    def get_next_url(self):
        max_inlink_url = max(self.list,key=self.list.get)
        del self.list[max_inlink_url]
        self.looked.add(max_inlink_url)
        return max_inlink_url

    def add_urls(self, urls):
        for url in urls:

            if url not in self.looked:

                if url not in self.list:
                    self.list[url] = 0

            self.list[url] += 1


# **Metodos auxiliares**

In [4]:
from urllib.request import urlopen, Request
import urllib.error
import validators
import time



# header com user agent para fazer as requisições
header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

#função para pegar o html
def get_html_page(url, delay):
    
    # valida se a string está em um formato de url
    if validators.url(url):

        try:
            time.sleep(delay)
            req = Request(url,headers=header)
            page = urlopen(req, timeout=5)

        # tramento dos erros
        except urllib.error.HTTPError as err:
            # save the http an the erro but still running
            print('\n' + url + ': ' + str(err.getcode()))
            return None
        except urllib.error.URLError as err:
            # save the url an the erro but still running
            print('\n' + url + ': ' + str(err.args))
            return None
        except socket.timeout:
            print("socket timeout")

        else:
            #checa se é um html
            if page.headers.get_content_type() == "text/html":
                return BeautifulSoup(page, "html.parser")
            else:
                return None
    else:
        return None

#extrai os links da pagina html
def extract_links(page, root, robot):
    robot.read()
    links = []

    for link in page.findAll('a', href=True):
        href = link.get('href')
        # print(href)
        if not re.search("^https:", href):
            href = root + href
        if re.search(rf"^{root}", href):
            if robot.can_fetch("*", href):
                links.append(href)
    return links

#salva o html
def save_html(page, root, count):
    
    html = str(page)
    file = open(root[8:len(root)-1] + "/" + str(count) + '.html',"w")
    file.write(html)
    file.close





# **Crawler bot**

In [5]:

class Bot:
    def __init__(self, root, euristica):
        self.root = root

        #iniciando a fronteira de de acordo com euristica
        if euristica:
            self.frontier = FrontierInlink(root)
        else:
            self.frontier = Frontier(root)
        
        #definindo um parser para o robot.txt
        self.robot = urobot.RobotFileParser()
        self.robot.set_url(root + 'robots.txt')

    def run(self, delay, n_pages):
        
        qt_pages = n_pages
        count = 1

        #roda até coletar uma quantidade de paginas ou a fronteira acabar
        while qt_pages > 0 and len(self.frontier.list) > 0:
            
            #pega proximo da fronteira
            url = self.frontier.get_next_url()
            
            #tenta coletar o html 
            page = get_html_page(url, delay)
            
            #se conseguir coletar (se não deu erro)
            if page is not None:

                #extrai 
                urls = extract_links(page, self.root, self.robot)
                self.frontier.add_urls(urls)
                save_html(page, self.root, count)
                count += 1
                qt_pages -= 1

        # print(self.root +' ok')
        return self.root + ': terminado'

# **Main**

In [6]:
def main():
  bots = []
  for url in urls:
    bots.append(Bot(url, False))

  with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for bot in bots:
      futures.append(executor.submit(bot.run, delay=delay, n_pages=qt_paginas, ))
    for future in concurrent.futures.as_completed(futures):
      print(future.result())


if __name__ == "__main__":
    main()


https://www.mercadolivre.com.br///www.mercadolivre.com.br: 404

https://www.shockgames.com.br/javascript:;: 418

https://www.shopb.com.br/javascript:;: 418

https://www.shopb.com.br/javascript:;: 418
https://www.shockgames.com.br/: terminado
https://www.ibyte.com.br/: terminado
https://www.shopb.com.br/: terminado
https://www.vnsgames.com.br/: terminado
https://www.mercadolivre.com.br/: terminado
https://www.bigboygames.com.br/: terminado
